In [1]:
import os
import glob
import wandb
#import pysdtw
import pickle
import warnings
import numpy as np
import torch as T
from tqdm import tqdm
from torch.nn import MSELoss
from itertools import compress
from utilities.multi_loss import MultiTaskLoss
from utilities.util import colate_fn as colate
from model import encoder, generator, smoothing
from utilities.util import TrTstSplit, GetInputOutputSplit

In [2]:
warnings.filterwarnings('ignore')
device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

pkls = ['*_lstm.pkl', '*_graph.pkl', '*_graph_mfc.pkl', '*_graph_pitch.pkl', '*_graph_eg.pkl']
losses = 'mse'

In [3]:
#create the lstm and gnn models 
LSTM_model = T.nn.Sequential(encoder.LSTM(29, 256, 2),
                             generator.LSTM(256, 2, 3),
                             smoothing.LearnableGaussianSmoothing(5)).to(device)
GCN_model = T.nn.Sequential(encoder.GCN(88, 256),
                            generator.GCN(256, 3),
                            smoothing.LearnableGaussianSmoothing(5)).to(device)
models = {'LSTM':LSTM_model,'graph':GCN_model}

In [4]:
def compute_pcc(y, y_hat):
    #print(y.shape,y_hat.shape)
    mean_true = T.mean(y,dim=0)
    mean_pred = T.mean(y_hat,dim=0)

    # Compute Pearson correlation coefficient
    numerator = T.sum((y_hat - mean_pred) * (y - mean_true),dim=0)
    denominator = T.sqrt(T.sum((y_hat - mean_pred)**2,dim=0)) * T.sqrt(T.sum((y - mean_true)**2,dim=0))
    correlation_coefficient = numerator / denominator

    # Clip correlation coefficient to prevent NaNs in gradients
    correlation_coefficient = T.clamp(correlation_coefficient, -1.0, 1.0)

    # Convert correlation coefficient to correlation loss (1 - r)
    correlation_loss = 1.0 - correlation_coefficient
    return T.mean(correlation_loss)

def pccloss(y, y_hat, lens):
    #print(y.shape,y_hat.shape)
    pearson = []
    
    curr = -1
    while True:
        #print(lens[0,curr])
        if lens[0,curr]==0:
            break
        if curr==-1:
            correlation_loss = compute_pcc(y[lens[0,curr]:,:], y_hat[lens[0,curr]:,:])
        else:
            correlation_loss = compute_pcc(y[lens[0,curr]:curr+1,:], y_hat[lens[0,curr]:curr+1,:])
        curr = lens[0,curr]-1
        pearson.append(correlation_loss)
    loss = T.mean(T.tensor(pearson))
    loss.requires_grad=True
    return loss

def cosineloss(y, y_hat, lens):
    #print(y.shape,y_hat.shape)
    pearson = []
    
    curr = -1
    while True:
        #print(lens[0,curr])
        if lens[0,curr]==0:
            break
        if curr==-1:
            correlation_loss = compute_cosine(y[lens[0,curr]:,:], y_hat[lens[0,curr]:,:])
        else:
            correlation_loss = compute_cosine(y[lens[0,curr]:curr+1,:], y_hat[lens[0,curr]:curr+1,:])
        curr = lens[0,curr]-1
        pearson.append(correlation_loss)
    loss = T.mean(T.tensor(pearson))
    loss.requires_grad=True
    return loss

def compute_cosine(y, y_hat):
    cos = T.nn.CosineSimilarity(dim=1, eps=1e-6)
    cosine = cos(y, y_hat)
    return 1-T.mean(cosine)

def lossmse(y, y_hat):
    mse = MSELoss()
    
    return mse(y,y_hat)

In [5]:
import sys
import torch_geometric as pyg
dirs = glob.glob('processed_data/'+pkls[4])
input = []
output = []
for cv in range(len(dirs)):
    train = []
    test = []
    for i in range(len(dirs)):
        with open(dirs[i], 'rb') as f:
            data = pickle.load(f)
        if 'Session2' in dirs[i]:
            for c in [41, 62, 69, 69, 73, 80, 100, 122, 150, 154, 374]:
                del data[c]
        if i == cv:
            test = data
        else:
            train.extend(data)
    model = models['graph']
    optimizer = T.optim.Adam(model.parameters(), lr=0.001)
    scheduler = T.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5,
                        patience=50, threshold=0.01, threshold_mode='rel',
                        verbose=True)

    dataloader = pyg.loader.DataLoader(train, batch_size=64, num_workers=4)
    dataloaderT = pyg.loader.DataLoader(test, batch_size=1)
    combiloss = MultiTaskLoss()
    epoch_loss = 0
    epoch_loss_pcc = 0
    epoch=0
    epochs=tqdm(total = 500)
    while epoch<500 and optimizer.param_groups[0]['lr']>0.000001:
        model.train()
        running_loss = 0.0
        running_loss_pcc = 0.0
        for data in dataloader:
            optimizer.zero_grad()

            with T.set_grad_enabled(True):
                outputs = model([data.x.to(device), data.edge_index.to(device)])
                loss = lossmse(outputs, data.y.to(device))
                #loss2 = cosineloss(data.y.to(device), outputs.squeeze(), data.edge_index.to(device))
                #loss = combiloss(loss,loss2)
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item()
            #running_loss_pcc += loss2.item()
        epoch_loss = running_loss / len(dataloader)
        #epoch_loss_pcc = running_loss_pcc / len(dataloader)
        scheduler.step(epoch_loss)# + epoch_loss_pcc)
        epoch+=1
        epochs.update(1)
        epochs.set_postfix(mse_loss=epoch_loss)#, pcc_loss=epoch_loss_pcc )
        
    
    model.eval()
    for data in dataloaderT:
        outputs = model([data.x.to(device), data.edge_index.to(device)])
        input.append(data.y.detach().cpu().numpy())
        output.append(outputs.detach().cpu().numpy())
dump = {'name':'graph_eg_mse','input':input,'output':output}
with open('results/'+'graph_eg_mse.pkl', 'wb') as f:
    pickle.dump(dump, f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()


100%|██████████| 500/500 [15:23<00:00,  1.82s/it, mse_loss=39.8]

# This notebook is for Graph Inter Experiments

In [6]:
outputs.shape, data.y.shape

(torch.Size([1, 7006, 3]), torch.Size([0]))

In [3]:
base_dir = '../IEMOCAP_full_release/'
os.chdir(base_dir)

In [5]:
from time import time
import torch_geometric as pyg
from utilities.graphData import MocapDataset as MocapDataset
GCN_model = T.nn.Sequential(encoder.GCN(512, 256),
                            generator.GCN(256, 3),
                            smoothing.LearnableGaussianSmoothing(5)).to(device)
param_count = sum(p.numel() for p in GCN_model.parameters() if p.requires_grad)
len=0
dirs = glob.glob('*.csv')
head, aud, landM, emo, val = GetInputOutputSplit([dirs[1]])
dataset = MocapDataset(head, aud, landM, emo, val)
dataloader = pyg.loader.DataLoader(dataset, batch_size=1 )
GCN_model.eval()
start = time()
for data in dataloader:
    outputs = GCN_model([data.x.to(device), data.edge_index.to(device)])
    len+=data.x.shape[0]
tick = time()-start
print(param_count,len/tick)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3320211 1423.9486234789726


In [5]:
len/tick

1384.4568916184305